### Import stuff

In [26]:
import os
import numpy as np
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

### Write a custom Dataloader

In [27]:
def to_int(x):
    if (not type(x) is str and (x==None or np.isnan(x))):
        return 32;
    return ord(x)

class AsciiToDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, height=40, width=80):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.height = height
        self.width = width
        
    def __len__(self):
        return len(self.img_labels)


    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        # create empty array
        arr = np.zeros([40,80], dtype="int")

        # indexes
        i, j = 0, 0

        # iterate through txt file and convert
        with open(img_path) as f:
            for line in f.readlines():
                for ch in line: 
                    if not (ch == "\n"):# leave out newline char
                        arr[j][i] = to_int(ch)
                    i += 1
                j += 1
                i = 0
        
        label = self.img_labels.iloc[idx, 1]
        return arr, label

### test dataset

In [28]:
annotation_file = "dataset/ascii.csv"
img_dir = "dataset/w80_h40_ascii"

test_dataset = AsciiToDataset(annotation_file, img_dir)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False) #shuffle=false to debug

train_features, train_labels = next(iter(test_dataloader))

In [29]:
print(train_features[0].shape)
print(train_features[0])

torch.Size([40, 80])
tensor([[64, 64, 64,  ..., 64, 64, 64],
        [64, 64, 64,  ..., 64, 64, 64],
        [64, 64, 64,  ..., 64, 64, 64],
        ...,
        [64, 64, 64,  ..., 64, 64, 64],
        [64, 64, 64,  ..., 64, 64, 64],
        [64, 64, 64,  ..., 64, 64, 64]])


### Sanity checks

In [30]:
img_dir = "dataset/w80_h40_ascii/109-ascii-art.txt"

with open(img_dir) as f:
    lines = f.readlines()
lines

['@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@@@#%%%@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@#=====*@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@#---::-+#@@@@@@@@@@@@@@@@@@@@@@@@@@@*-:+@@@@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@%=---::::.-+@@@@@@@@@@@@@@@@@@@@@@@#.     .+@@@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@##=::::--...-*@@@@@@@@@@@@@@@@@@@@%         :*@@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@+::..::.....=@@@@@@@@@@@@@@@@@@@=           +@@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@%+=+-........+@@@@@@@@@@@@@@@@@@. .....      #@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@@@@@-     ....#@@@@@@@@@@@@@@@@%..:::::.. .  *@@@@@@@@@@@@@@@\n',
 '@@@@@@@@@@@@@@@@@@@@@@%.    ....::#@@@@@@